## MySQL 
In this notebook, we created a ZomatoDB database, designed to store restaurants information, evaluations, and ratings.
This data was collected from Zomato API. Exploratory_Data_Analysis.ipynb contains basic exploratory data analaysis of this dataset.
Our approach consists on creating different relational tables using MySQL Workbench and PyMySQL. Then, we populated these tables and run analysis to test the database functionality. 

### Database creation:
We locally host a MySQL database. 

In [31]:
import pymysql
import pymysql.cursors
from itertools import chain

# Connect to the database that we ran in our localhost
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='34006503'
                            )

#From our connection we need a cursor, which acts as our interface into the database
cur = connection.cursor()
# We can verity we are connected:
print(connection)

In [29]:
# Create ZomatoDB database:
cur.execute("CREATE DATABASE ZomatoDB")

1

In [2]:
# Verify that the database was created
cur.execute('SHOW DATABASES')
for db in cur:
    print(db)

('information_schema',)
('mysql',)
('performance_schema',)
('sys',)
('testdb',)
('ZomatoDB',)


In [32]:
# Then we specify a connection to the ZomatoDB database:
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='34006503',
                            database='ZomatoDB')

#From our connection we need a cursor, which acts as our interface into the database
cur = connection.cursor()

In [112]:
# Start cleaning all tables that already exists:
cur.execute("DROP TABLES country_code, cuisines, evaluation, locality, main_table, ratings")
connection.commit()

### Tables creation:

In [113]:
# Create main table that store information about a restaurant's name, id, and location:
cur.execute("DROP TABLE IF EXISTS main_table;")
connection.commit()

create_main_table = """CREATE TABLE main_table(
                        Restaurant_ID INT NOT NULL,
                        Restaurant_Name VARCHAR(255),
                        Country_Code INT,
                        City VARCHAR(255),
                        Address VARCHAR(255),
                        Locality VARCHAR(255),
                        Longitude DOUBLE,
                        Latitude DOUBLE,
                        KEY (Country_Code),
                        PRIMARY KEY (Restaurant_ID));"""
                            
cur.execute(create_main_table)
connection.commit()

In [114]:
## Cuisine table consists of the different types of cuisines that you can find in each restaurant:
cur.execute("DROP TABLE IF EXISTS cuisines")
connection.commit()

create_cuisines_table = """CREATE TABLE cuisines(
                            Restaurant_ID INT,
                            Cuisine_type VARCHAR(255),
                            KEY (Restaurant_ID));"""

cur.execute(create_cuisines_table)
connection.commit()

In [115]:
## Country_codes table is an additional table that we created to make the country_code column useful:
cur.execute("DROP TABLE IF EXISTS country_code")
connection.commit()

create_countrycode_table = """CREATE TABLE country_code(
                            country_code INT,
                            Country VARCHAR(255),
                            PRIMARY KEY (country_code));"""

cur.execute(create_countrycode_table)
connection.commit()

In [116]:
# Evaluation table store the results of restaurants evaluations, which consisted on yes/no cuestions
# regarding certain characteristics:
cur.execute("DROP TABLE IF EXISTS evaluation;")
connection.commit()

create_evaluation_table = """CREATE TABLE evaluation(
                                Restaurant_ID INT,
                                Table_booking BOOL,
                                Online_delivery BOOL,
                                Now_delivering BOOL,
                                Switch_menu BOOL,
                                Price_range INT,
                                KEY (Restaurant_ID)
                                );"""
cur.execute(create_evaluation_table)
connection.commit()

In [117]:
# ratings table store the results of the restaurants ratings:
cur.execute("DROP TABLE IF EXISTS ratings;")
connection.commit()

create_rating_table = """CREATE TABLE ratings(
                            Restaurant_ID INT,
                            Rating FLOAT, 
                            Rating_color VARCHAR(255),
                            Rating_text VARCHAR(255),
                            Votes DOUBLE,
                            KEY (Restaurant_ID));"""

cur.execute(create_rating_table)
connection.commit()

In [118]:
cur.execute("DROP TABLE IF EXISTS locality;")
connection.commit()

create_locality_table = """CREATE TABLE locality(
                            Restaurant_ID INT,
                            Locality VARCHAR(255),
                            KEY (Restaurant_ID));
                            """

cur.execute(create_locality_table)
connection.commit()


In [119]:
# Verify that tables were created:
cur.execute('SHOW TABLES')
for tb in cur:
    print(tb)

('country_code',)
('cuisines',)
('evaluation',)
('locality',)
('main_table',)
('ratings',)


### Migration of data:

In [82]:
# First we imported the dataset that came in .csv format:
import pandas as pd
import numpy as np

zomato_df = pd.read_csv('zomato.csv', encoding='latin1')

# We also imported a dataset that help us to map the original Country code
# to its name:
code_df = pd.read_excel('Country-Code.xlsx')

In [83]:
# We drop NaN values, edited the yes/no evaluation columns to booleans, and
# partitioned the dataset into simplier dataframes tha will help us to populate

# the different tables in the database:
zomato_df.dropna(inplace=True)

# Yes/No to boolean:
d = {'Yes': True, 'No':False}
zomato_df['Has Table booking'] = zomato_df['Has Table booking'].map(d)
zomato_df['Has Online delivery'] = zomato_df['Has Online delivery'].map(d)
zomato_df['Is delivering now'] = zomato_df['Is delivering now'].map(d)
zomato_df['Switch to order menu'] = zomato_df['Switch to order menu'].map(d)

# We also need to transform the column Restaurant ID to integer type:
zomato_df['Restaurant ID'] = zomato_df['Restaurant ID'].astype(int)

# main_table data:
main_table_df = zomato_df[[
    'Restaurant ID', 
    'Restaurant Name', 
    'Country Code', 
    'City', 'Address', 'Locality', 
    'Longitude', 'Latitude']]

# evaluation table data:
evaluation_df = zomato_df[[
    'Restaurant ID',
    'Has Table booking',
    'Has Online delivery',
    'Is delivering now',
    'Switch to order menu',
    'Price range']]
# ratings table data:
ratings_df = zomato_df[[
    'Restaurant ID',
    'Aggregate rating',
    'Rating color',
    'Rating text',
    'Votes']]


In [96]:
main_table_df[main_table_df.duplicated()]

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Longitude,Latitude


In [85]:
# We saw in the Exploratory Data analysis notebook that cuisines column
# has nested lists of the different cuisine types. We dissagregate those lists
# and stored them in new rows:
# return list from series of comma-separated strings
def chainer(s):
  return list(chain.from_iterable(s.str.split(',')))

# calculate lengths of splits
lens = zomato_df['Cuisines'].str.split(',').map(len)

# create new dataframe, reapeting or chaining as appropiate
res = pd.DataFrame({
    'Restaurant ID': np.repeat(zomato_df['Restaurant ID'], lens),
    'Cuisines':chainer(zomato_df['Cuisines'])
})

res_cuisine = res.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
res_cuisine.head(3)

,Restaurant ID,Cuisines
0,6317637,French
0,6317637,Japanese
0,6317637,Desserts


In [86]:
# Likewise, we saw that Locality column contains nested list of the localities
# of each restaurants. Therefore, we dissagregated those cells into new columns:
# calculate lengths of splits
lens_locality = zomato_df['Locality'].str.split(',').map(len)

# create new dataframe, reapeting or chaining as appropiate
res_first = pd.DataFrame({
    'Restaurant ID': np.repeat(zomato_df['Restaurant ID'], lens_locality),
    'Locality':chainer(zomato_df['Locality'])
})

res_locality = res_first.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
res_locality.head(3)

,Restaurant ID,Locality
0,6317637,Century City Mall
0,6317637,Poblacion
0,6317637,Makati City


We have now partitioned the original dataset into:
- main_table_df -> basic information of restaurants like name, id, location.
- evaluation_df -> evaluation yes/no questions, transformed to booleans.
- ratings_df -> rating of each restaurant.
- code_df -> countries code.
- res_cuisine -> different types of restaurants cuisines

To migrate the data we transformed our dataframes into tuples, and then 
inserted these tuples into their respective MySQL tables:

In [87]:
# transform the different dataframes to tuples:
main_tuples = [tuple(x) for x in main_table_df.to_numpy()]
evaluation_tuples = [tuple(x) for x in evaluation_df.to_numpy()]
rating_tuples = [tuple(x) for x in ratings_df.to_numpy()]
cuisine_tuples = [tuple(x) for x in res_cuisine.to_numpy()]
locality_tuples = [tuple(x) for x in res_locality.to_numpy()]
country_code_tuples = [tuple(x) for x in code_df.to_numpy()]



In [120]:
# Then we created sql formulas to be executed in our cursor:
main_sql = """INSERT INTO main_table 
                    (
                    Restaurant_ID,
                    Restaurant_Name,
                    Country_Code,
                    City,
                    Address,
                    Locality,
                    Longitude,
                    Latitude) 
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s);""" #%s placeholders

evaluation_sql = """INSERT INTO evaluation 
                    (
                    Restaurant_ID,
                    Table_booking,
                    Online_delivery,
                    Now_delivering,
                    Switch_menu,
                    Price_range) 
                    VALUES(%s, %s, %s, %s, %s, %s);"""

rating_sql = """INSERT INTO ratings 
                    (
                    Restaurant_ID,
                    Rating,
                    Rating_color,
                    Rating_text,
                    Votes) 
                    VALUES(%s, %s, %s, %s, %s);"""

cuisine_sql = """INSERT INTO cuisines
                    (
                    Restaurant_ID,
                    Cuisine_type) 
                    VALUES (%s, %s);"""

locality_sql = """INSERT INTO locality
                    (
                    Restaurant_ID,
                    Locality) 
                    VALUES (%s, %s);"""

country_code_sql = """INSERT INTO country_code
                    (
                    country_code,
                    Country) 
                    VALUES (%s, %s);"""



In [121]:
# Execute all above:
cur.executemany(main_sql, main_tuples)
connection.commit()

cur.executemany(evaluation_sql, evaluation_tuples)
connection.commit()

cur.executemany(rating_sql, rating_tuples)
connection.commit()

cur.executemany(cuisine_sql, cuisine_tuples)
connection.commit()

cur.executemany(locality_sql, locality_tuples)
connection.commit()

cur.executemany(country_code_sql, country_code_tuples)
connection.commit()

In [122]:
# Now we create the raltion of the tables with Restaurant_ID KEY:

# cuisines table:
cur.execute("""ALTER TABLE cuisines
                ADD CONSTRAINT cuisine_fk FOREIGN KEY (Restaurant_ID) 
                    REFERENCES main_table(Restaurant_ID);""")
connection.commit()

# evaluation:
cur.execute("""ALTER TABLE evaluation
                ADD CONSTRAINT evaluation_fk FOREIGN KEY (Restaurant_ID) 
                    REFERENCES main_table(Restaurant_ID);""")
connection.commit()

# ratings:
cur.execute("""ALTER TABLE ratings
                ADD CONSTRAINT ratings_fk FOREIGN KEY (Restaurant_ID) 
                    REFERENCES main_table(Restaurant_ID);""")
connection.commit()

# locality:
cur.execute("""ALTER TABLE locality
                ADD CONSTRAINT locality_fk FOREIGN KEY (Restaurant_ID) 
                    REFERENCES main_table(Restaurant_ID);""")
connection.commit()

# country_code:
cur.execute("""ALTER TABLE country_code
                ADD CONSTRAINT country_fk FOREIGN KEY (country_code) 
                    REFERENCES main_table(Country_Code);""")
connection.commit()

In [127]:
res_cuisine[res_cuisine['Cuisines'] == 'American']

,Restaurant ID,Cuisines
11,6309455,American
15,18185059,American
19,6308205,American
20,6315438,American
21,6310406,American
...,...,...
9504,5800746,American
9507,5800156,American
9511,5800316,American
9513,5800433,American
